In [1]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')

# Resend alerts on tickets that remain OPEN or in WORKING state for TOO LONG
print("============================================")
print("/  EscalateTicket is running.               /")
print("============================================")

import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser

config = configparser.ConfigParser()
config.read("../../../analytics_secrets.ini")

_SLACK_TOKEN = config['slack']['token']
_CHIRPSTACK_USER = config['chirpstack']['user']
_CHIRPSTACK_PASS = config['chirpstack']['password']

_DB_HOST = config['kanjidb']['dbhost']
_DB_PORT = config['kanjidb']['dbport']
_DB_NAME = config['kanjidb']['dbname']
_DB_USER = config['kanjidb']['dbuser']
_DB_PASS  = config['kanjidb']['dbpass']

_LOG_DEBUG = 0
_LOG_INFO  = 1
_LOG_ERROR = 2
_LOG_LEVEL = int(config['DEFAULT']['loglevel'])
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, _SLACK_TOKEN)
logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

/  EscalateTicket is running.               /
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [2]:
def postMessageToSlack(blockmessage):    
    sc = SlackClient(_SLACK_TOKEN)
    slackchannel = "infrastructure"
    response = sc.api_call("chat.postMessage", channel=slackchannel, blocks=blockmessage)
    print(blockmessage) 
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(blockmessage)
      print(response)
    else:
      print("Ok posting message to Slack channel")    

In [6]:
_LOG_LEVEL = _LOG_DEBUG

# Reissue Alerts on OPEN tickets every 15minutes
_TICKETAGE_REISSUE_THRESHOLD_SECONDS = 15 * 60

_SLACK_TICKET_CHANNEL = "infrastructure"

#Ticket Status
_TICKET_STATUS_OPEN    = 10000
_TICKET_STATUS_WORKING = 10001

ticketnow = datetime.now(pytz.utc)  #tz Aware
now = datetime.now() + timedelta(hours = 4)

ticketquery = "SELECT * FROM kanji_ticket WHERE status_id={} OR status_id={}".format(_TICKET_STATUS_OPEN, _TICKET_STATUS_WORKING)
logger(_LOG_DEBUG, ticketquery)
df = pd.read_sql(ticketquery, conn)

logger(_LOG_DEBUG, "number of tickets OPEN or WORKING {}".format(len(df.index))) 
for ind in df.index:
    idticket = df['idticket'][ind]
    ticketts = df['ticketts'][ind]
    description = df['description'][ind]
    _SLACK_CHANNEL = df['ticketchannel'][ind]
    nodeid = df['node_id'][ind]
    nodequery = "SELECT node.name, location.description, location.imageurl, location.slackchannel, \
                 customer.slacktoken FROM kanji_node node \
                 JOIN kanji_location location ON node.location_id=location.idlocation \
                 JOIN kanji_customer customer ON node.customer_id=customer.idcustomer \
                 WHERE idnode={}".format(nodeid)
    df2 = pd.read_sql(nodequery, conn)
    nodename = df2["name"][0]
    locationimageurl = df2["imageurl"][0]
    locationdescription = df2["description"][0]
    _SLACK_TOKEN = df2["slacktoken"][0]
    ticketage = ticketnow - df['lastupdatetimestamp'][ind]
    ticketageseconds =  ticketage.days*24*60*60 + ticketage.seconds
    if ticketageseconds >= _TICKETAGE_REISSUE_THRESHOLD_SECONDS:
      logger(_LOG_DEBUG, "")
      logger(_LOG_DEBUG, nodename)
      logger(_LOG_DEBUG, locationimageurl)
      logger(_LOG_DEBUG, _SLACK_TOKEN)
      logger(_LOG_DEBUG, _SLACK_CHANNEL)
      logger(_LOG_DEBUG, idticket )
      mentions = "@Charlie"
      message = "*REISSUE ALERT* - " + description
      ts = kt.slackticket(nodename, locationdescription, message, mentions, 2, 3, \
                        locationimageurl, _SLACK_TOKEN, _SLACK_CHANNEL, idticket, ticketts)
      if ts!=0:
        kt.updateTicket(conn, idticket, ticketts)
        logger(_LOG_INFO, "Alert reissued for ticket {}.".format(idticket))
    else:
      status = "Ok"
      #is there an OPEN ticket on this mote?
    
  

SELECT * FROM kanji_ticket WHERE status_id=10000 OR status_id=10001
number of tickets OPEN or WORKING 4
